In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from google.colab import files

In [ ]:
# Upload your model and image files to Colab
print("Please upload your TensorFlow Lite model file:")
uploaded = files.upload()
MODEL_PATH = list(uploaded.keys())[0]  # Get the uploaded model path


Saving vww_96_grayscale_quantized.tflite to vww_96_grayscale_quantized.tflite


In [ ]:
print("Please upload an image file for testing:")
uploaded = files.upload()
TEST_IMAGE_PATH = list(uploaded.keys())[0]  # Get the uploaded image path

In [ ]:
# Initialize the TensorFlow Lite interpreter
try:
    interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
    interpreter.allocate_tensors()
except AttributeError as e:
    print("Error initializing the TensorFlow Lite Interpreter. Ensure you have a compatible TensorFlow version.")
    exit(1)

In [ ]:
# Get input and output details of the model
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']


In [ ]:
# Function to load and preprocess an image for model input
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (input_shape[1], input_shape[2]))
    image = np.expand_dims(image, axis=-1).astype(np.float32) / 255.0  # Normalize to [0,1]
    image = np.expand_dims(image, axis=0)  # Model expects a 4D tensor
    return image

In [ ]:
# Function to classify a single image
def classify_image(image_path):
    image = load_and_preprocess_image(image_path)
    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # Find the predicted label and confidence
    predicted_label = np.argmax(output_data)
    confidence = output_data[0][predicted_label]

    return predicted_label, confidence


In [ ]:
# Function to classify a single image and return confidence percentages
def classify_image(image_path):
    image = load_and_preprocess_image(image_path)
    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])[0]

    # Calculate confidence percentages
    dog_confidence = round(float(output_data[0]) * 100, 2)
    cat_confidence = round(float(output_data[1]) * 100, 2)

    return {"Dog": dog_confidence, "Cat": cat_confidence}


In [ ]:
# Main function to classify an image
def main():
    result = classify_image(TEST_IMAGE_PATH)
    print(f"Classification result for {TEST_IMAGE_PATH}:")
    print(f"Dog: {result['Dog']}%")
    print(f"Cat: {result['Cat']}%")

if __name__ == "__main__":
    main()

Classification result for dog.4006.jpg:
Dog: 98.55%
Cat: 1.45%
